In [1]:
# Imports
import inspect
import random
import tempfile
import uuid
from functools import wraps
import collections

import coloredlogs
import dask as dd
import logging
import numpy as np
import pandas as pd
from coolname import generate_slug

# Logger
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG')

# StackOverflow snippet #1
########################################################################################################################
# /////|   Decorator   |////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def auto_assign_arguments(function):

    @wraps(function)
    def wrapped(self, *args, **kwargs):
        _assign_args(self, list(args), kwargs, function)
        function(self, *args, **kwargs)

    return wrapped


########################################################################################################################
# /////|   Utils   |////////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def _assign_args(instance, args, kwargs, function):

    def set_attribute(instance, parameter, default_arg):
        if not (parameter.startswith("_")):
            setattr(instance, parameter, default_arg)

    def assign_keyword_defaults(parameters, defaults):
        for parameter, default_arg in zip(reversed(parameters),
                                          reversed(defaults)):
            set_attribute(instance, parameter, default_arg)

    def assign_positional_args(parameters, args):
        for parameter, arg in zip(parameters, args.copy()):
            set_attribute(instance, parameter, arg)
            args.remove(arg)

    def assign_keyword_args(kwargs):
        for parameter, arg in kwargs.items():
            set_attribute(instance, parameter, arg)

    def assign_keyword_only_defaults(defaults):
        return assign_keyword_args(defaults)

    def assign_variable_args(parameter, args):
        set_attribute(instance, parameter, args)

    POSITIONAL_PARAMS, VARIABLE_PARAM, _, KEYWORD_DEFAULTS, _, KEYWORD_ONLY_DEFAULTS, _ = inspect.getfullargspec(
        function)
    POSITIONAL_PARAMS = POSITIONAL_PARAMS[1:]  # remove 'self'

    if (KEYWORD_DEFAULTS):
        assign_keyword_defaults(parameters=POSITIONAL_PARAMS,
                                defaults=KEYWORD_DEFAULTS)
    if (KEYWORD_ONLY_DEFAULTS):
        assign_keyword_only_defaults(defaults=KEYWORD_ONLY_DEFAULTS)
    if (args): assign_positional_args(parameters=POSITIONAL_PARAMS, args=args)
    if (kwargs): assign_keyword_args(kwargs=kwargs)
    if (VARIABLE_PARAM):
        assign_variable_args(parameter=VARIABLE_PARAM, args=args)


########################################################################################################################


########################################################################################################################
# /////|   Class   |////////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################
class initSim:
    """Class to initialize the progenitor (F0) population.

    initSim primarily creates an object which stores information about the 
    initial population. Moreover the inbuilt methods will return pandas
    dataframe for direct modification as desired.
    
    :return: Object
    :rtype: object type initSim
    """

    @auto_assign_arguments
    def __init__(self,
                 tcount=1,
                 ttype=[1],
                 popsize=100,
                 insize=2000,
                 gensize=1000,
                 trate=[0.02],
                 tpenalty=[-0.02],
                 tparent=["Mother"]):
        """Class constructor
        :param tcount: Number of transposons to be present in initial population
        :param ttype: Type of transposons (class 1 or 2)
        :param popsize: Population size
        :param insize: Number of insertion sites
        :param gensize: Size of genome on cM
        :param trate: Transposition rates
        :param tpenalty: Selection penalty for each transposon
        :param tparent: Parent carrying the transposon insertion (Mother/Father)
        :type tcount: int
        :type ttype: list [int]
        :type popsize: int
        :type insize: int
        :type gensize: int
        :type trate: list [float]
        :type tpenalty: list [float]
        :type arg1: tparent [string]
        """
        # Dataframe definations
        self.TranspFrame = pd.DataFrame(columns=[
            'TID', 'Position', 'SelCo', 'Name', 'Class', 'Traceback',
            'Generation', "Parent", "TraRate"
        ])
        self.PopFrame = pd.DataFrame(columns=[
            'PID', 'Fitness', 'Name', 'Sex', 'Lineage', 'Generation', 'TE',
            'Insertion_Father', 'Insertion_Mother'
        ])

    # Init transposons
    def initT(self):
        """Method to dataframe containing initial transposon population
        
        :return: Dataframe containing Transposon information
        :rtype: Dataframe
        """

        if (self.tcount > len(self.tpenalty)):
            logger.info(
                "Mismatch between transposon count and selection penalties. Using default for each transposon count!"
            )
            self.tpenalty = [-0.02] * self.tcount
        if (self.tcount > len(self.trate)):
            logger.info(
                "Mismatch between transposon count and transposition rates. Using default for each transposon count!"
            )
            self.trate = [0.02] * self.tcount
        if (self.tcount > len(self.ttype)):
            logger.info(
                "Mismatch between transposon count and transposon types. Using default for each transposon count!"
            )
            self.ttype = [1] * self.tcount
        if (self.tcount > len(self.tparent)):
            logger.info(
                "Mismatch between transposon count and transposon parent. Using default for each transposon count!"
            )
            self.tparent = ["Mother"] * self.tcount

        # Create random filled insertion sites
        inSiteArray = random.sample(range(1, self.insize), self.tcount)

        for i in range(0, self.tcount):
            row = pd.Series({
                'TID': uuid.uuid4().hex,
                'Position': inSiteArray[i],
                'TraRate': self.trate[i],
                'SelCo': self.tpenalty[i],
                'Name': generate_slug(),
                'Class': self.ttype[i],
                'Traceback': ['0'],
                'Generation': 1,
                'Parent': self.tparent[i]
            })
            self.TranspFrame = self.TranspFrame.append(row, ignore_index=True)
        return (self.TranspFrame)

    # Init population and genome
    def initPG(self):
        """Method to create initial population and their respective genomes
        
        :return: tuple(population,genome)
            WHERE
            population is population dataframe
            genome is genome dataframe
        :rtype: DataFrame
        """

        # Create transposon insertions in randomly selected individuals
        IndividualToInsert = random.sample(list(range(1, self.popsize)),
                                           self.tcount)
        TIDlist = self.TranspFrame.TID.tolist()
        TIDcounter = 0
        Parent = "0"
        insertion_Father = 0
        insertion_Mother = 0
        FitnessPen = 0
        for i in range(self.popsize):
            # In case this (un)lucky individual has transposon insertion
            if (i in IndividualToInsert):
                TE = TIDlist[TIDcounter]
                TIDcounter += 1
                Parent = self.TranspFrame[self.TranspFrame['TID'] ==
                                          TE]['Parent'].values[0]
                FitnessPen = self.TranspFrame[self.TranspFrame['TID'] ==
                                              TE]['SelCo'].values[0]
                if (Parent == "Mother"):
                    insertion_Mother = self.TranspFrame[
                        self.TranspFrame['TID'] == TE]['Position'].values[0]

                if (Parent == "Father"):
                    insertion_Father = self.TranspFrame[
                        self.TranspFrame['TID'] == TE]['Position'].values[0]

            else:
                TE = '0'
                Parent = "0"
                insertion_Father = 0
                insertion_Mother = 0

            # Populate the population!
            rowPop = pd.Series({
                'PID': uuid.uuid4().hex,
                'Fitness': random.uniform(0.6, 1.0),
                'Name': generate_slug(),
                'Sex': 'H',
                'Lineage': ['0'],
                'Generation': 1,
                'TE': [TE],
                'Insertion_Father': [insertion_Father],
                'Insertion_Mother': [insertion_Mother]
            })
            self.PopFrame = self.PopFrame.append(rowPop, ignore_index=True)

        self.PopFrame['Lineage'] = self.PopFrame['Lineage'].astype('object')
        self.PopFrame['TE'] = self.PopFrame['TE'].astype('object')
        self.PopFrame['Insertion_Father'] = self.PopFrame[
            'Insertion_Father'].astype('object')
        self.PopFrame['Insertion_Mother'] = self.PopFrame[
            'Insertion_Mother'].astype('object')
        return (self.PopFrame)

    def createSim(self):
        """Method to generate the initial simulation dataset
        
        :return: List
            WHERE 
            index 0 is transposon dataframe
            index 1 is population dataframe
            index 2 is genome dataframe
        :rtype: list
        """

        transposon = self.initT()
        genome = self.initPG()
        return ([transposon, genome])


########################################################################################################################

Genome = pd.DataFrame(
    columns=['Position', 'RecombinationRate', 'InsertionSite'])


def generateGenome(genomeSize, nInsertionSites, nChrom):
    genome = list(range(1, genomeSize + 1))
    InsertionSite = [0] * len(genome)
    RecombinationRate = [0.01] * len(genome)
    insertionLocation = np.random.choice(genomeSize,
                                         nInsertionSites,
                                         replace=False)
    chrLocation = np.random.choice(genomeSize, nChrom, replace=False)

    insertionCounter = 1
    for i in insertionLocation:
        InsertionSite[i] = insertionCounter
        insertionCounter += 1

    insertionCounter = 1
    for i in chrLocation:
        if (InsertionSite[i] != 0):
            RecombinationRate[i + 1] = 0.5
            insertionCounter += 1
        else:
            RecombinationRate[i] = 0.5
            insertionCounter += 1

    genomeDict = {
        'Position': genome,
        'RecombinationRate': RecombinationRate,
        'InsertionSite': InsertionSite
    }
    genome = pd.DataFrame(genomeDict)
    return (genome)

In [114]:
def recombination(genomeFrame, genomeOrg):
    # Create insertion list for the progeny
    TEprogeny = []
    # Create a copy of genomeFrame
    genomeCopy = genomeFrame.copy(deep=True)
    Insertion_Father = genomeOrg['Insertion_Father']
    Insertion_Mother = genomeOrg['Insertion_Mother']
    if (Insertion_Father[0] == 0 and Insertion_Mother[0] == 0):
        return (random.choice(
            [genomeOrg['Insertion_Father'], genomeOrg['Insertion_Mother']]))
    else:
        initParent = random.choice(["M", "F"])
        surrogateParent = "M" if ("M" != initParent) else "F"
        RandArray = np.random.uniform(0, 1.0, genomeFrame.shape[0])
        switch = genomeCopy['RecombinationRate'] > RandArray
        #counter = collections.Counter(switch)
        #print(counter)
        genomeCopy['Progeny'] = np.where(switch.cumsum() % 2 == 0, initParent,
                                         surrogateParent)
        for i in Insertion_Father:
            se = genomeCopy[genomeCopy['InsertionSite'] ==
                            i]['Progeny'].values[0]
            if (se == "M"):
                TEprogeny.append(i)
        for i in Insertion_Mother:
            se = genomeCopy[genomeCopy['InsertionSite'] ==
                            i]['Progeny'].values[0]
            if (se == "F"):
                TEprogeny.append(i)
        #print(genomeCopy['Progeny'].value_counts())
    return (TEprogeny)

In [119]:
genome = generateGenome(100000, 2000, 100)

In [120]:
k = initSim(tcount=80)
t = k.createSim()
t[1]

2019-11-29 15:53:48 siddharth-OptiPlex-7070 __main__[7760] INFO Mismatch between transposon count and selection penalties. Using default for each transposon count!
2019-11-29 15:53:48 siddharth-OptiPlex-7070 __main__[7760] INFO Mismatch between transposon count and transposition rates. Using default for each transposon count!
2019-11-29 15:53:48 siddharth-OptiPlex-7070 __main__[7760] INFO Mismatch between transposon count and transposon types. Using default for each transposon count!
2019-11-29 15:53:48 siddharth-OptiPlex-7070 __main__[7760] INFO Mismatch between transposon count and transposon parent. Using default for each transposon count!


,PID,Fitness,Name,Sex,Lineage,Generation,TE,Insertion_Father,Insertion_Mother
0,518a1a7a74ba44e99e7648de2159857c,0.783604,hungry-super-sunfish-of-imagination,H,[0],1,[0],[0],[0]
1,c61c343c2beb46049a74c0496adcb446,0.791367,accelerated-lively-doberman-of-peace,H,[0],1,[0],[0],[0]
2,0783ac6512394287842219ee64cb2ece,0.840823,amazing-denim-labradoodle-of-serendipity,H,[0],1,[0d9e05237d4e447c8b4640ff0aba76cf],[0],[240]
3,0de9d187c55d49b5b23d95dc5db064ea,0.656587,optimistic-viridian-partridge-of-glee,H,[0],1,[0],[0],[0]
4,973e98e7b38948f882a517d4cd74edf4,0.806923,busy-defiant-quoll-of-enterprise,H,[0],1,[4c36c45fcbeb440a8410d3ee49a9a155],[0],[1354]
...,...,...,...,...,...,...,...,...,...
95,7c77e80bf16c4f258c80bd01809332e1,0.715979,dainty-quick-peccary-of-performance,H,[0],1,[9b6a6e76ae664e63b225e266b651bcf5],[0],[1631]
96,47e735805a8b46c9a08ba88d61d7611d,0.935211,courageous-optimal-nuthatch-of-excellence,H,[0],1,[5f530ae10fa84fa8a51453d69bc1a690],[0],[1207]
97,1b824ddfcc3c436ca659d331e7350f81,0.987678,orthodox-weightless-buffalo-from-nibiru,H,[0],1,[49e3256c36994092b3ab4b4f41c00e02],[0],[1465]
98,2af126d314934c3d8922dd5e497347bc,0.715210,wine-taipan-of-impossible-aptitude,H,[0],1,[b1be196fd40246a2bfcb6f3ffbaa5467],[0],[1707]


In [123]:
%%timeit
for i in (range(t[1].shape[0])):
    recombination(genome, t[1].iloc[i])

536 ms ± 7.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


M
F
